In [ ]:

import numpy as np
# Set a seed for reproducibility
np.random.seed(42)

# define time series generation 

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from statsmodels.formula.api import ols
from stargazer.stargazer import Stargazer
from IPython.display import display, HTML
from plotly.subplots import make_subplots


class TimeSeriesGenerator:
    def __init__(self, T=1000, P_0=100, Q_0=100,
                 seasonal_diff_price=5, std_noise=0.1,
                 betaRTP=-0.2, betaExpRTP=-0.2, Q_base = 95, beta_seasonality=10,
                 seasonality_func=None):
        self.T = T
        self.P_0 = P_0
        self.Q_0 = Q_0
        self.Q_base = Q_base
        self.seasonal_diff_price = seasonal_diff_price
        self.std_noise = std_noise
        self.betaRTP = betaRTP
        self.betaExpRTP = betaExpRTP
        self.beta_seasonality = beta_seasonality
        self.seasonality_func = seasonality_func or (lambda idx: np.where(idx % 2 == 0, 'even', 'odd'))
        self.df = self.generate_time_series()

    def generate_time_series(self):
        index = np.arange(self.T)
        seasonality = self.seasonality_func(index)  # Apply the custom seasonality function
        P_diff = self.seasonal_diff_price
        P_noise = np.random.normal(0, self.P_0 * self.std_noise, self.T)
       
        actual_price = np.where(seasonality == 'peak', self.P_0 + P_diff, self.P_0 - P_diff) + P_noise
        expected_price = np.where(seasonality == 'peak', self.P_0 + P_diff, self.P_0 - P_diff)
       
        df = pd.DataFrame({
            'index': index,
            'seasonality': seasonality,
            'actual_price': actual_price,
            'expected_price': expected_price
        })
        return df

    def add_fixed_quantity(self, std_error=None):
        if std_error is None:
            std_error = self.std_noise * self.Q_0
        fixed_noise = np.random.normal(0, std_error, self.T)
        self.df['load_without_response'] = self.Q_0 + fixed_noise

    def add_actual_RTP_quantity(self, betaRTP=None, std_error=None):
        if betaRTP is None:
            betaRTP = self.betaRTP
        if std_error is None:
            std_error = self.std_noise * self.Q_0
        RTP_noise = np.random.normal(0, std_error, self.T)
        self.df['load_actual_price_repsonse'] = self.Q_0 + betaRTP * (self.df['actual_price'] - self.P_0 ) + RTP_noise

    def add_expected_RTP_quantity(self, betaExpRTP=None, std_error=None):
        if betaExpRTP is None:
            betaExpRTP = self.betaExpRTP
        if std_error is None:
            std_error = self.std_noise * self.Q_0
        pseudoRTP_noise = np.random.normal(0, std_error, self.T)
        self.df['load_expected_price_repsonse'] = self.Q_0 + betaExpRTP * (self.df['expected_price'] - self.P_0) + pseudoRTP_noise
 
    def add_seasonal_quantity(self, beta_seasonality=None, std_error=None):
        if beta_seasonality is None:
            beta_seasonality = self.beta_seasonality
        if std_error is None:
            std_error = self.std_noise * self.Q_0
        seasonal_noise = np.random.normal(0, std_error, self.T)
        seasonality_numeric = np.where(self.df['seasonality'] == 'peak', 1, 0)
        self.df['seasonal_load_without_response'] = self.Q_base + beta_seasonality * seasonality_numeric + seasonal_noise

    def add_seasonal_actual_RTP_quantity(self, betaRTP=None, beta_seasonality=None, std_error=None):
        if betaRTP is None:
            betaRTP = self.betaRTP
        if beta_seasonality is None:
            beta_seasonality = self.beta_seasonality 
        if std_error is None:
            std_error = self.std_noise * self.Q_0
        seasonal_RTP_noise = np.random.normal(0, std_error, self.T)
        seasonality_numeric = np.where(self.df['seasonality'] == 'peak', 1, 0)
        self.df['seasonal_load_actual_price_repsonse'] = self.Q_base +betaRTP * (self.df['actual_price'] - self.P_0)+ beta_seasonality * seasonality_numeric + seasonal_RTP_noise

    def add_seasonal_expected_RTP_quantity(self, betaExpRTP=None, beta_seasonality=None, std_error=None):
        if betaExpRTP is None:
            betaExpRTP = self.betaExpRTP
        if beta_seasonality is None:
            beta_seasonality = self.beta_seasonality
        if std_error is None:
            std_error = self.std_noise * self.Q_0
        seasonal_expectedRTP_noise = np.random.normal(0, std_error, self.T)
        seasonality_numeric = np.where(self.df['seasonality'] == 'peak', 1, 0)
        self.df['seasonal_load_expected_price_repsonse'] = self.Q_base + betaExpRTP * (self.df['expected_price'] - self.P_0) + beta_seasonality * seasonality_numeric + seasonal_expectedRTP_noise




    def plot_correlations(self):
        # Convert categorical variables to dummy variables
        df_with_dummies = pd.get_dummies(self.df, drop_first=True)
       
        # Compute the correlation matrix
        corr_matrix = df_with_dummies.corr()

        # Generate a mask for the upper triangle
        mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
        upper_corr_matrix = corr_matrix.where(mask)

        # Plot a heatmap using Plotly for interactivity
        fig = px.imshow(upper_corr_matrix, text_auto=True, color_continuous_scale='RdBu_r', aspect='auto')
        fig.update_layout(title='Interactive Correlation Matrix', xaxis_title='Variables', yaxis_title='Variables')
        fig.show()




    # def plot_scatter(self, x_variable, y_variable, trendline='ols', trendline_scope=None):
    #     if x_variable not in self.df.columns or y_variable not in self.df.columns:
    #         print(f"One or both variables '{x_variable}' and '{y_variable}' are not in the DataFrame.")
    #         return

    #     if trendline_scope is None:
    #         fig = px.scatter(self.df, x=x_variable, y=y_variable, trendline=trendline,
    #                         labels={x_variable: x_variable, y_variable: y_variable},
    #                         title=f'Scatter Plot of {y_variable} vs. {x_variable} with Regression Line for all data',
    #                         opacity=0.5)
    #     else:
    #         fig = px.scatter(self.df, x=x_variable, y=y_variable, color=trendline_scope, trendline=trendline,
    #                         labels={x_variable: x_variable, y_variable: y_variable},
    #                         title=f'Scatter Plot of {y_variable} vs. {x_variable}',
    #                         opacity=0.5)

    #     fig.show()

    # def plot_scatter(self, x_variable, y_variable, trendline='ols', trendline_scope=None, return_fig=False):
    #     if x_variable not in self.df.columns or y_variable not in self.df.columns:
    #         print(f"One or both variables '{x_variable}' and '{y_variable}' are not in the DataFrame.")
    #         return

    #     # Colorblind-friendly palette
    #     colorblind_palette = ["#0072B2", "#D55E00", "#CC79A7", "#999999", "#E69F00", "#56B4E9", "#F0E442", "#009E73"]

    #     fig = px.scatter(
    #         self.df, x=x_variable, y=y_variable,
    #         color=trendline_scope if trendline_scope else None,
    #         trendline=trendline,
    #         labels={x_variable: x_variable, y_variable: y_variable},
    #         title=f'Scatter Plot of {y_variable} vs. {x_variable}',
    #         opacity=0.5,
    #         color_discrete_sequence=colorblind_palette,
    #         template="simple_white"
    #     )

    #     # Update marker styles
    #     if trendline_scope is not None:
    #         # Set the marker symbol based on group
    #         unique_groups = self.df[trendline_scope].unique()
    #         symbols = ["circle", "triangle-up"]
    #         for trace in fig.data:
    #             if trace.marker.symbol == "circle":
    #                 trace.update(marker=dict(symbol=symbols[0]))
    #             else:
    #                 trace.update(marker=dict(symbol=symbols[1]))

    #     # Update trendline styles
    #     for trace in fig.data:
    #         if trace.mode == 'lines':
    #             # Make trendlines thicker and black
    #             trace.update(line=dict(width=3, color='black'))
    #             if trendline_scope is not None:
    #                 # Update pattern by group
    #                 trace.update(line=dict(dash='dash' if trace.name == unique_groups[0] else 'dot'))

    #     # Ensure trendlines are on top by reorganizing traces
    #     fig.data = [trace for trace in fig.data if trace.mode != 'lines'] + [trace for trace in fig.data if trace.mode == 'lines']

    #     if return_fig:
    #         return fig
    #     else:
    #         fig.show()


    # def plot_2x2_subplots(self, subplot_specs):
    #     fig = make_subplots(
    #         rows=2, cols=2,
    #         shared_xaxes=True, shared_yaxes=True,
    #         subplot_titles=[spec.get('title', '') for spec in subplot_specs],
    #         vertical_spacing=0.1,  # Reduce spacing between rows
    #         horizontal_spacing=0.05  # Adjust if you want to change horizontal spacing
    #     )

    #     for i, spec in enumerate(subplot_specs):
    #         row = i // 2 + 1
    #         col = i % 2 + 1
    #         scatter_fig = self.plot_scatter(
    #             x_variable=spec.get('x_variable'),
    #             y_variable=spec.get('y_variable'),
    #             trendline=spec.get('trendline', 'ols'),
    #             trendline_scope=spec.get('trendline_scope'),
    #             return_fig=True
    #         )

    #         for trace in scatter_fig.data:
    #             trace.showlegend = trace.showlegend and (i == 0)
    #             fig.add_trace(trace, row=row, col=col)

    #     # Ensure x and y axes have the same scale
    #     fig.update_yaxes(scaleanchor="x", scaleratio=1)
    #     fig.update_xaxes(matches='x')  # This makes all x-axes have the same scale

    #     # Update layout for better visualization
    #     fig.update_layout(
    #         height=800, width=1200,
    #        # title_text="2x2 Subplots of Scatter Plots with Regression Lines",
    #         margin=dict(t=40, b=40, l=40, r=40),  # Adjust margins if needed
    #         showlegend = True, 
    #         legend = dict(
    #             orientation = 'h', 
    #             x = 0.5, 
    #             xanchor = 'center', 
    #             y = -0.01#
    #         )
    #     )
    #     fig.show()

   






# generate time series 

In [ ]:
# Example usage
ts_gen = TimeSeriesGenerator(T=10000, std_noise=0.1, betaRTP=-0.2, betaExpRTP=-0.2, seasonal_diff_price = 10, beta_seasonality=-0.0,
                            seasonality_func = lambda idx: np.where((np.arange(len(idx)) % 10) > 5, 'peak', 'base'))

ts_gen.add_fixed_quantity()
ts_gen.add_actual_RTP_quantity()
ts_gen.add_expected_RTP_quantity()
ts_gen.add_seasonal_quantity()
ts_gen.add_seasonal_actual_RTP_quantity()
ts_gen.add_seasonal_expected_RTP_quantity()


# analyse time series


## plot over time 

In [ ]:
def plot_timeseries(df, variables):
    fig = go.Figure()
    T= len(df)
    # TODO: outside of class 
    # Add all price series to the same plot
    for variable in variables: 
        fig.add_trace(go.Scatter(x=df['index'], y=df[variable], mode='lines', name=variable))

    # fig.add_trace(go.Scatter(x=df['index'], y=df['load_without_response'], mode='lines', name='load without Response'))
    # fig.add_trace(go.Scatter(x=df['index'], y=df['load_actual_price_repsonse'], mode='lines', name='load with Response to Actual Price'))
    # fig.add_trace(go.Scatter(x=df['index'], y=df['load_expected_price_repsonse'], mode='lines', name='load with Response to Expected Price'))
    # fig.add_trace(go.Scatter(x=df['index'], y=df['seasonal_load_without_response'], mode='lines', name='Seasonal load without Response'))
    
    # Update layout for better visualization
    fig.update_layout(
        title=f'Variables over time (T = {T})',
        xaxis_title='Time',
        yaxis_title='Value (unitless)',
        hovermode='x unified'
        
    )
    
    fig.show()
variables = ['actual_price', 'expected_price']
plot_timeseries(ts_gen.df, variables)

## get the amount of variance explained by the seasonality 

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm


# Fit the OLS model using the formula API
model = sm.formula.ols(formula='actual_price ~ seasonality', data=ts_gen.df).fit()

# Print the regression results
print(model.summary())

## boxplots

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Assume ts_gen.df is your DataFrame
# Print the first few rows of the DataFrame to verify
# Create a list of columns to plot
columns_to_plot_price = ['actual_price']
columns_to_plot_quantity = ['load_actual_price_repsonse', 'load_expected_price_repsonse','load_without_response']#, 'seasonal_load_without_response', 'seasonal_load_actual_price_repsonse','seasonal_load_expected_price_repsonse' ]

# Combine all columns for easy iteration
columns_to_plot = columns_to_plot_price + columns_to_plot_quantity

# Calculate common y-axis limits
combined_data = pd.concat([ts_gen.df[column] for column in columns_to_plot])
common_ylim = (combined_data.min() - 0.1 * abs(combined_data.min()), combined_data.max() + 0.1 * abs(combined_data.max()))

# Initialize the matplotlib figure with a suitable size and gridspec
fig = plt.figure(figsize=(24, 6))
gs = fig.add_gridspec(1, len(columns_to_plot_price) + len(columns_to_plot_quantity), width_ratios=[2] * len(columns_to_plot_price) + [1] * len(columns_to_plot_quantity), wspace=0.4)

# Loop through the columns and create a boxplot for each by seasonality
for i, column in enumerate(columns_to_plot):
    ax = fig.add_subplot(gs[i])
    sns.boxplot(x='seasonality', y=column, data=ts_gen.df, ax=ax)
    
    # Calculate the mean for each seasonality group
    means = ts_gen.df.groupby('seasonality')[column].mean()
    
    # Plot the mean as a white circle with a black edge
    for j, season in enumerate(means.index):
        ax.scatter([j], [means[season]], color='white', edgecolor='black', zorder=10)
    
    ax.set_title(f'{column}')
    ax.set_xlabel('Seasonality')
    ax.set_ylabel(column)
    ax.set_ylim(common_ylim)

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()

# Regression

In [ ]:
def estimate_price_reaction_with_ols_with_stargazer_output(df, models, output = 'html'):
   # Ensure dependent variable is in the dataframe

   # Perform regressions
   results = []
   for model in models:
      result = ols(model, data=df).fit()
      results.append(result)

   # Combine results using stargazer
   stargazer = Stargazer(results)
   stargazer.title("Regression Results")
   stargazer.custom_columns(models, [1] * len(models))  # Dynamically adjust number of columns
   stargazer.show_model_numbers(True)

   # Render HTML in Jupyter Notebook
   if output == 'latex':
      print(stargazer.render_latex())
   else:
      display(HTML(stargazer.render_html()))

## regression coefficients

In [ ]:



models = [
   # f"{y} ~ seasonality",
    f"load_actual_price_repsonse ~ actual_price",
    f"load_expected_price_repsonse ~ actual_price",
    f"load_actual_price_repsonse ~ actual_price + seasonality",
    f"load_expected_price_repsonse ~ actual_price  + seasonality",

]
estimate_price_reaction_with_ols_with_stargazer_output(df = ts_gen.df, models = models, output = 'html')
estimate_price_reaction_with_ols_with_stargazer_output(df = ts_gen.df, models = models, output = 'latex')




## Visualizaiton of regression results

In [ ]:
# # Create the 2x2 subplot with customized specifications
# subplot_specs = [
#     {'x_variable': 'actual_price', 'y_variable': 'load_actual_price_repsonse', 'trendline': 'ols', 'title': "load Reacting to Actual Price"},
#     {'x_variable': 'actual_price', 'y_variable': 'load_actual_price_repsonse', 'trendline': 'ols', 'trendline_scope': 'seasonality', 'title': "load Reacting to Actual Price by Seasonality"},
#     {'x_variable': 'actual_price', 'y_variable': 'load_expected_price_repsonse', 'trendline': 'ols', 'title': "load Reacting to Expected Price"},
#     {'x_variable': 'actual_price', 'y_variable': 'load_expected_price_repsonse', 'trendline': 'ols', 'trendline_scope': 'seasonality', 'title': "load Reacting to Expected Price by Seasonality"},

# ]

# ts_gen.plot_2x2_subplots(subplot_specs)

In [ ]:
def plot_scatter(df, x_variable, y_variable, trendline='ols', trendline_scope=None, return_fig=False):
    if x_variable not in df.columns or y_variable not in df.columns:
        print(f"One or both variables '{x_variable}' and '{y_variable}' are not in the DataFrame.")
        return

    # Colorblind-friendly palette
    colorblind_palette = ["#0072B2", "#D55E00", "#CC79A7", "#999999", "#E69F00", "#56B4E9", "#F0E442", "#009E73"]

    fig = px.scatter(
        df, x=x_variable, y=y_variable,
        color=trendline_scope if trendline_scope else None,
        trendline=trendline,
        labels={x_variable: x_variable, y_variable: y_variable},
        title=f'Scatter Plot of {y_variable} vs. {x_variable}',
        opacity=1000/len(df[x_variable])*0.9,
        color_discrete_sequence=colorblind_palette,
        template="simple_white"
    )

    # Update marker styles
    if trendline_scope is not None:
        # Set the marker symbol based on group
        unique_groups = df[trendline_scope].unique()
        symbols = ["circle", "triangle-up"]
        for trace in fig.data:
            if trace.marker.symbol == "circle":
                trace.update(marker=dict(symbol=symbols[0]))
            else:
                trace.update(marker=dict(symbol=symbols[1]))

    # Update trendline styles
    for trace in fig.data:
        if trace.mode == 'lines':
            # Make trendlines thicker and black
            trace.update(line=dict(width=3, color='black'))
            if trendline_scope is not None:
                # Update pattern by group
                trace.update(line=dict(dash='dash' if trace.name == unique_groups[0] else 'dot'))

    # Ensure trendlines are on top by reorganizing traces
    fig.data = [trace for trace in fig.data if trace.mode != 'lines'] + [trace for trace in fig.data if trace.mode == 'lines']

    if return_fig:
        return fig
    else:
        fig.show()


def plot_2x2_subplots(df, subplot_specs):
    fig = make_subplots(
        rows=2, cols=2,
        shared_xaxes=True, shared_yaxes=True,
        subplot_titles=[spec.get('title', '') for spec in subplot_specs],
        vertical_spacing=0.1,  # Reduce spacing between rows
        horizontal_spacing=0.05  # Adjust if you want to change horizontal spacing
    )

    for i, spec in enumerate(subplot_specs):
        row = i // 2 + 1
        col = i % 2 + 1
        scatter_fig =plot_scatter(df,
            x_variable=spec.get('x_variable'),
            y_variable=spec.get('y_variable'),
            trendline=spec.get('trendline', 'ols'),
            trendline_scope=spec.get('trendline_scope'),
            return_fig=True
        )

        for trace in scatter_fig.data:
            trace.showlegend = trace.showlegend and (i == 0)
            fig.add_trace(trace, row=row, col=col)

    # Ensure x and y axes have the same scale
    fig.update_yaxes(scaleanchor="x", scaleratio=1)
    fig.update_xaxes(matches='x')  # This makes all x-axes have the same scale

    # Update layout for better visualization
    fig.update_layout(
        height=800, width=1200,
        # title_text="2x2 Subplots of Scatter Plots with Regression Lines",
        margin=dict(t=40, b=40, l=40, r=40),  # Adjust margins if needed
        showlegend = True, 
        legend = dict(
            orientation = 'h', 
            x = 0.5, 
            xanchor = 'center', 
            y = -0.01#
        )
    )
    fig.show()


In [ ]:
# Create the 2x2 subplot with customized specifications
subplot_specs = [
    {'x_variable': 'actual_price', 'y_variable': 'load_actual_price_repsonse', 'trendline': 'ols', 'title': "load Reacting to Actual Price"},
    {'x_variable': 'actual_price', 'y_variable': 'load_actual_price_repsonse', 'trendline': 'ols', 'trendline_scope': 'seasonality', 'title': "load Reacting to Actual Price by Seasonality"},
    {'x_variable': 'actual_price', 'y_variable': 'load_expected_price_repsonse', 'trendline': 'ols', 'title': "load Reacting to Expected Price"},
    {'x_variable': 'actual_price', 'y_variable': 'load_expected_price_repsonse', 'trendline': 'ols', 'trendline_scope': 'seasonality', 'title': "load Reacting to Expected Price by Seasonality"},

]

plot_2x2_subplots(ts_gen.df, subplot_specs)

# Seasonal demand

In [ ]:
# Example usage
ts_gen = TimeSeriesGenerator(T=10000, std_noise=0.05, betaRTP=-0.2, betaExpRTP=-0.2,Q_0 = 100, Q_base= 95, seasonal_diff_price = 10, beta_seasonality=10,
                            seasonality_func = lambda idx: np.where((np.arange(len(idx)) % 10) > 5, 'peak', 'base'))

ts_gen.add_fixed_quantity()
ts_gen.add_actual_RTP_quantity()
ts_gen.add_expected_RTP_quantity()
ts_gen.add_seasonal_quantity()
ts_gen.add_seasonal_actual_RTP_quantity()
ts_gen.add_seasonal_expected_RTP_quantity()


In [ ]:
variables = ['actual_price', 'expected_price', 
'load_without_response', 'load_actual_price_repsonse', 'load_expected_price_repsonse', 
'seasonal_load_without_response', 'seasonal_load_actual_price_repsonse', 'seasonal_load_expected_price_repsonse']
plot_timeseries(ts_gen.df, variables)


In [ ]:


models = [
   # f"{y} ~ seasonality",
    f"seasonal_load_actual_price_repsonse ~ actual_price",
    f"seasonal_load_expected_price_repsonse ~ actual_price",
    f"seasonal_load_actual_price_repsonse ~ actual_price + seasonality",
    f"seasonal_load_expected_price_repsonse ~ actual_price  + seasonality",

]
estimate_price_reaction_with_ols_with_stargazer_output(df = ts_gen.df, models = models, output = 'html')
estimate_price_reaction_with_ols_with_stargazer_output(df = ts_gen.df, models = models, output = 'latex')


# Control groups

In [ ]:
ts_gen.df.columns

In [ ]:

models = [
   # f"{y} ~ seasonality",

    f"seasonal_load_actual_price_repsonse ~ actual_price + seasonality +  seasonal_load_without_response", # the seasonal load without response is only there to capture any additional effects. Otherwise not needed 
    f"seasonal_load_expected_price_repsonse ~ actual_price + seasonality +  seasonal_load_without_response", # the seasonal load without response is only there to capture any additional effects. Otherwise not needed 

    # f"seasonal_load_expected_price_repsonse ~  seasonal_load_without_response",
    # f"seasonal_load_actual_price_repsonse ~ actual_price + seasonal_load_without_response",
    # f"seasonal_load_expected_price_repsonse ~ actual_price + seasonal_load_without_response",
    # f"seasonal_load_actual_price_repsonse ~ actual_price + seasonality + seasonal_load_without_response",
    # f"seasonal_load_expected_price_repsonse ~ actual_price  + seasonality + seasonal_load_without_response",
    # f"seasonal_load_expected_price_repsonse ~ expected_price + seasonality ",


]
estimate_price_reaction_with_ols_with_stargazer_output(df = ts_gen.df, models = models, output = 'html')
estimate_price_reaction_with_ols_with_stargazer_output(df = ts_gen.df, models = models, output = 'latex')

### comparing estimate for control group with estimate of treatment group

In [ ]:
models = [
   # f"{y} ~ seasonality",

    f"seasonal_load_actual_price_repsonse ~ actual_price ", # the seasonal load without response is only there to capture any additional effects. Otherwise not needed 
    f"seasonal_load_expected_price_repsonse ~ actual_price ", # the seasonal load without response is only there to capture any additional effects. Otherwise not needed 

    f"seasonal_load_without_response ~ actual_price", # the seasonal load without response is only there to capture any additional effects. Otherwise not needed 

    f"seasonal_load_actual_price_repsonse ~ actual_price + seasonality", # the seasonal load without response is only there to capture any additional effects. Otherwise not needed 
    f"seasonal_load_expected_price_repsonse ~ actual_price + seasonality", # the seasonal load without response is only there to capture any additional effects. Otherwise not needed 

    f"seasonal_load_without_response ~ actual_price+ seasonality", # the seasonal load without response is only there to capture any additional effects. Otherwise not needed 

    # f"seasonal_load_expected_price_repsonse ~  seasonal_load_without_response",
    # f"seasonal_load_actual_price_repsonse ~ actual_price + seasonal_load_without_response",
    # f"seasonal_load_expected_price_repsonse ~ actual_price + seasonal_load_without_response",
    # f"seasonal_load_actual_price_repsonse ~ actual_price + seasonality + seasonal_load_without_response",
    # f"seasonal_load_expected_price_repsonse ~ actual_price  + seasonality + seasonal_load_without_response",
    # f"seasonal_load_expected_price_repsonse ~ expected_price + seasonality ",


]
estimate_price_reaction_with_ols_with_stargazer_output(df = ts_gen.df, models = models, output = 'html')
estimate_price_reaction_with_ols_with_stargazer_output(df = ts_gen.df, models = models, output = 'latex')

### reaction to expected price

In [ ]:
models = [
   # f"{y} ~ seasonality",

    f"seasonal_load_actual_price_repsonse ~ expected_price ", # the seasonal load without response is only there to capture any additional effects. Otherwise not needed 
    f"seasonal_load_expected_price_repsonse ~ expected_price ", # the seasonal load without response is only there to capture any additional effects. Otherwise not needed 

    f"seasonal_load_without_response ~ expected_price", # the seasonal load without response is only there to capture any additional effects. Otherwise not needed 

    f"seasonal_load_actual_price_repsonse ~ expected_price + seasonality", # the seasonal load without response is only there to capture any additional effects. Otherwise not needed 
    f"seasonal_load_expected_price_repsonse ~ expected_price + seasonality", # the seasonal load without response is only there to capture any additional effects. Otherwise not needed 

    f"seasonal_load_without_response ~ expected_price+ seasonality", # the seasonal load without response is only there to capture any additional effects. Otherwise not needed 

    # f"seasonal_load_expected_price_repsonse ~  seasonal_load_without_response",
    # f"seasonal_load_actual_price_repsonse ~ actual_price + seasonal_load_without_response",
    # f"seasonal_load_expected_price_repsonse ~ actual_price + seasonal_load_without_response",
    # f"seasonal_load_actual_price_repsonse ~ actual_price + seasonality + seasonal_load_without_response",
    # f"seasonal_load_expected_price_repsonse ~ actual_price  + seasonality + seasonal_load_without_response",
    # f"seasonal_load_expected_price_repsonse ~ expected_price + seasonality ",


]
estimate_price_reaction_with_ols_with_stargazer_output(df = ts_gen.df, models = models, output = 'html')
estimate_price_reaction_with_ols_with_stargazer_output(df = ts_gen.df, models = models, output = 'latex')

## Panel estimation 

In [ ]:
import numpy as np
import pandas as pd

import pyfixest as pf




#### fixest example

In [ ]:
data = pf.get_data()
pf.feols("Y ~ X1 | f1 + f2", data=data).summary()

In [ ]:
# OLS Estimation: estimate multiple models at once
fit = pf.feols("Y + Y2 ~X1 | csw0(f1, f2)", data = data, vcov = {'CRV1':'group_id'})
# Print the results
fit.etable()

In [ ]:
fit1 = fit.fetch_model(0)
fit1.vcov("hetero").summary()

In [ ]:

fit.vcov("hetero").etable()

In [ ]:
fit_iv = pf.feols("Y ~ 1 | f1 | X1 ~ Z1", data = data)
fit_iv.summary()

#### generate data

In [ ]:
ts_gen.df.columns

In [ ]:
def generate_panel_data(ts_gen, consumer_type, fe = 0):

    ts_gen.add_fixed_quantity()
    ts_gen.add_actual_RTP_quantity()
    ts_gen.add_expected_RTP_quantity()
    ts_gen.add_seasonal_quantity()
    ts_gen.add_seasonal_expected_RTP_quantity()
    ts_gen.add_seasonal_actual_RTP_quantity()
    
    df = ts_gen.df[['index', 'seasonality', 'actual_price', 'expected_price', consumer_type]].copy()
    df['consumer_type'] = consumer_type
    df['FE']= fe
    df.rename(columns={consumer_type: 'load'}, inplace=True)

    return df


In [ ]:
# Example usage
ts_gen = TimeSeriesGenerator(T=1000, std_noise=0.05, betaRTP=-0.2, betaExpRTP=-0.2,Q_0 = 100, Q_base= 95, seasonal_diff_price = 10, beta_seasonality=10,
                            seasonality_func = lambda idx: np.where((np.arange(len(idx)) % 10) > 5, 'peak', 'base'))

In [ ]:
# panel estimation 

def estimated_panel_data(treatment, control, n, m, formula = 'load ~ consumer_type *actual_price | csw0(seasonality)'):
    # Initialize the FE counter
    fe_counter = 1

    # Initialize empty DataFrames
    df1 = pd.DataFrame()
    df2 = pd.DataFrame()


    # Append the data with unique FE values
    for i in range(n):
        df1 = pd.concat([df1, generate_panel_data(ts_gen,treatment, fe_counter)], ignore_index=True)
        fe_counter += 1

    for i in range(m):
        df2 = pd.concat([df2, generate_panel_data(ts_gen, control, fe_counter)], ignore_index=True)
        fe_counter += 1

    # Combine both DataFrames
    df_panel = pd.concat([df1, df2], ignore_index=True)


    # Convert consumer_type to category and set base category
    df_panel['consumer_type'] = df_panel['consumer_type'].astype('category')
    df_panel['consumer_type'] = df_panel['consumer_type'].cat.reorder_categories([control, treatment], ordered=True)
    df_panel['consumer_type'] = df_panel['consumer_type'].cat.set_categories([control, treatment], ordered=True)
    # 'type1' will be used as the base (reference) category.

    fit = pf.feols(formula, data = df_panel,  vcov = {'CRV1':'FE'})
    return fit.etable()



In [ ]:
# Generate and append n times for seasonal_load_without_response and m times for seasonal_load_actual_price_repsonse
n = 100
m = 100


In [ ]:
treatment= 'seasonal_load_actual_price_repsonse'
control = 'seasonal_load_without_response'
estimated_panel_data(treatment, control, n, m, formula = 'load ~ consumer_type *actual_price | csw0(seasonality)')

In [ ]:

treatment= 'seasonal_load_actual_price_repsonse'
# control = 'seasonal_load_without_response'
control = 'seasonal_load_expected_price_repsonse'

estimated_panel_data(treatment, control, n, m, formula = 'load ~ consumer_type *actual_price | csw0(seasonality)')

In [ ]:
treatment = 'seasonal_load_expected_price_repsonse'
control = 'seasonal_load_without_response'

estimated_panel_data(treatment, control, n, m, formula = 'load ~ consumer_type *actual_price | csw0(seasonality)')

In [ ]:
treatment = 'load_actual_price_repsonse'
#control = 'load_without_response'

#control = 'seasonal_load_without_response'
control = 'seasonal_load_expected_price_repsonse'


estimated_panel_data(treatment, control, n, m, formula = 'load ~ consumer_type *actual_price *seasonality| csw0(FE)')

In [ ]:
treatment = 'load_expected_price_repsonse'

control = 'seasonal_load_without_response'

estimated_panel_data(treatment, control, n, m, formula = 'load ~ consumer_type *actual_price| csw0(seasonality)')


In [ ]:
estimated_panel_data(treatment, control, n, m, formula = 'load ~ consumer_type *actual_price *seasonality| csw0(seasonality)')

# Endogeneity

In [ ]:
# define a class called EquilibriumTimeSeriesGeneartor, which generate a time series C_t of lenght T (default T = 100) 
# (a capacitY) which may have a seasonality e.g. C_t = beta_seasonlity * Seasonality + epsionl_C; 
# the seasonality should be an external function, i.e. 
# seasonality_func = lambda idx: np.where((np.arange(len(idx)) % 10) > 5, 'high', 'low')
# based on this capacity, I would like to generated equilibrium quantities and equilbirum prices
# an equilbrium price is defined as the P_equilbrium_t that solves the followign equation: S_t (P_equilibrium_t)= Aggregtaed_Demand_t(P_queilibrium_t)
# the supply function: S_t (P_t) = S_0 + \beta_S * P_t + epsilon_s_t  + beta_capacity C_t  
# per default, beta_capacity should be 1; 
# \beta_seasonlity should be 0 if the the supply has no seasonality, and a a share of S_0 otherwise 
# there are two  demand funcitons (with the option to expend to more later) that form aggregate demand 
# D_aggregated_t = D_1_t + d_2_t
# D_1_t = D_10 + \beta_P1 *P_t + \epsilon_D1_t
# D_2_t = D_10 + \beta_P2 *P_expected_t + \epsilon_D2_t
# allow for the option to define P_expected_t, for now, set P_expected_t = P_t 
# # return the equilbrium quantities for D1 and D2 (and potentially more in the future) and the equilibrium price
# variables = ['P_eq', 'C_t','seasonality_generation','D_agg_t']
# plot_timeseries(df = generator.df, variables= variables)


## define data geneartion

#### without function to include expected load

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



class EquilibriumTimeSeriesGenerator:
    def __init__(self, T=100, S_0=0, beta_capacity=1, seasonal_generation_difference=100, seasonal_generation_variance = 10, beta_S=10, generation_noise_variance=1,random_generation_max  = 10,  seasonal_generation_function = None, linespace_expansion = 0):
        self.T = T
        self.S_0 = S_0
        self.beta_capacity = beta_capacity
        self.seasonal_generation_difference = seasonal_generation_difference
        self.seasonal_generation_function = seasonal_generation_function
        self.random_generation_max = random_generation_max
        self.beta_S = beta_S
        self.generation_noise_variance = generation_noise_variance
        self.demand_functions = []
        self.individual_noises = []  # To store noise series for each demand function
        self.seasonal_demand_series = []
        self.df = None
        self.linespace_expansion = linespace_expansion
        self.seasonal_generation_variance = seasonal_generation_variance



    # linear demand function with optional seasonality 
    def demand_func(self, P_t, d_0, beta_P, epsilon_D=0, seasonal_load_difference_t=0):
        return d_0 + beta_P * P_t + seasonal_load_difference_t + epsilon_D

    def add_demand_function(self, d_0, beta_P, noise_variance_demand, seasonal_load_difference=0, seasonal_load_function=None):
        """Add a custom demand function and handle seasonal adjustments."""
        # Generate demand noise
        demand_noise = np.random.normal(0, noise_variance_demand, self.T)
        # Generate seasonal demand series
        if seasonal_load_function is not None:
            seasonal_load = seasonal_load_function(np.arange(self.T))
            seasonal_demand = seasonal_load * seasonal_load_difference
        else:
            seasonal_demand = np.zeros(self.T)  # Default to no seasonal change if function is not provided
        # Append noise and seasonal demand series
        self.individual_noises.append(demand_noise)
        self.seasonal_demand_series.append(seasonal_demand)
        self.demand_functions.append(lambda P_t, seasonal_load_difference_t, epsilon_D, d_0=d_0, beta_P=beta_P: 
                                    self.demand_func(P_t, d_0, beta_P, seasonal_load_difference_t, epsilon_D))
        # TODO: Where is the link between sesaonal_load_differenc_t and sesaonal_laod function etc.? 
        # TODO: where is the link between demand_noise and epsilon_D? Is that acutally used? 
        # TODO: how is demand_noise used in the demand used for generating the data? Could it be that it is not used at all? 
        # TOOD or is that used later? once the data is generated? 

    def add_multiple_demand_functions(self, N, d_0, beta_P, noise_variance_demand, seasonal_load_difference=0, seasonal_load_function=None):
        """Add multiple custom demand functions at once, with optional seasonal adjustments."""
        for n in range(1, 1 + N):
            self.add_demand_function(d_0, beta_P,noise_variance_demand, seasonal_load_difference, seasonal_load_function)


    def generate_time_series(self):
        index = np.arange(self.T)
        P_eq = []
        S_t_values = []
        D_t_values = []
        individual_D_t_values = {f'D{i+1}_t': [] for i in range(len(self.demand_functions))}
        if self.seasonal_generation_function is not None:
            seasonality_generation = np.array([self.seasonal_generation_function(i) for i in range(self.T)])
        else:
            seasonality_generation = np.zeros(self.T)
        
        epsilon_c_t = np.random.random(self.T)
        epsilon_c_t_deterministic = np.random.normal(0,self.seasonal_generation_variance, self.T)
        C_t = self.seasonal_generation_difference * seasonality_generation + epsilon_c_t_deterministic * seasonality_generation + epsilon_c_t*self.random_generation_max
 
        aggregated_seasonality_load = np.zeros(self.T)

        # define grid 
        linespace_min = -100*(1+self.linespace_expansion)
        linespace_max = 100*(1+self.linespace_expansion)
        print(f'Search equilbrium in grid between {linespace_min} and {linespace_max}')
        steps = (linespace_max - linespace_min)*500
        print(steps)

        for t in range(self.T):
            P_t = np.linspace(linespace_min, linespace_max, steps)
            epsilon_s_t = np.random.normal(0, self.generation_noise_variance)

            def supply_function(P_t):
                return self.S_0 + self.beta_S * P_t + epsilon_s_t + self.beta_capacity * C_t[t]
                
            def demand_function(P_t):
                D_agg_t = np.zeros_like(P_t)
                time_step_demands = []
                for i, (demand_func, seasonal_demand_series, noise_series) in enumerate(zip(self.demand_functions, self.seasonal_demand_series, self.individual_noises)):
                    D_t_i = demand_func(P_t, seasonal_load_difference_t=seasonal_demand_series[t], epsilon_D=noise_series[t])  # Use pre-generated noise for time t
                    time_step_demands.append(D_t_i)
                    D_agg_t += D_t_i
                return D_agg_t, time_step_demands
            supply_values = supply_function(P_t)
            demand_values, individual_demands = demand_function(P_t)
            equilibrium_indices = np.where(np.isclose(supply_values, demand_values, atol=1))[0]
            if len(equilibrium_indices) > 0:
                middle_index = (len(equilibrium_indices) // 2)
                P_equilibrium = P_t[equilibrium_indices[middle_index]]
            else:
                P_equilibrium = np.nan
            P_eq.append(P_equilibrium)
            if np.isnan(P_equilibrium):
                S_t_values.append(np.nan)
                D_t_values.append(np.nan)
                for i in range(len(self.demand_functions)):
                    individual_D_t_values[f'D{i+1}_t'].append(np.nan)
            else:
                S_t_values.append(supply_function(P_equilibrium))
                D_t_values.append(demand_values[equilibrium_indices[middle_index]])
                for i in range(len(self.demand_functions)):
                    individual_D_t_values[f'D{i+1}_t'].append(individual_demands[i][equilibrium_indices[middle_index]])
            aggregated_seasonality_load[t] = sum([seasonal_load[t] for seasonal_load in self.seasonal_demand_series])
        for key in individual_D_t_values:
            individual_D_t_values[key] = np.array(individual_D_t_values[key])
        self.df = pd.DataFrame({
            'index': index,
            'seasonality_generation': seasonality_generation,
            'aggregated_seasonality_load': aggregated_seasonality_load,
            'C_t': C_t,
            'P_eq': P_eq,
            'S_t': S_t_values,
            'D_agg_t': D_t_values,
        })
        for key, values in individual_D_t_values.items():
            self.df[key] = values


    def plot_inverse_functions(self):
        P_t = np.linspace(-100, 100, 1000)

        def inverse_supply_function(P_t):
            return self.S_0 + self.beta_S * P_t

        def inverse_aggregate_demand_function(P_t):
            D_agg_t = np.zeros_like(P_t)
            for demand_func in self.demand_functions:
                D_agg_t += demand_func(P_t, epsilon_D=0)  # Pass epsilon_D=0 for plotting
            return D_agg_t

        supply_values = inverse_supply_function(P_t)
        demand_values = inverse_aggregate_demand_function(P_t)

        plt.figure(figsize=(10, 6))
        plt.plot(P_t, supply_values, label='Inverse Supply Function', color='blue')
        plt.plot(P_t, demand_values, label='Inverse Aggregate Demand Function', color='red')
        plt.xlabel('P_t')
        plt.ylabel('D_agg_t')
        plt.title('Inverse Supply and Aggregate Demand Functions')
        plt.legend()
        plt.grid(True)
        plt.show()



# Plot inverse functions
# ts_gen_eq.plot_inverse_functions()

# why is the aggregate demand higher in absolute terms than the joined? # because it is a linear response! 

# Example usage


# treatment = 'actual_price_repsonse'
# N_treatment = 10 # with response
# control = 'no_response'
# N_control = 10 # without response

# # load with response to actual price

# # Example usage
# ts_gen_eq = EquilibriumTimeSeriesGenerator(
#     T = 1000, 
#     generation_noise_variance=1, 
#     seasonal_generation_function = lambda t: 1 if (t % 3) == 1 else 0, # [0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, ... solar like 
#     seasonal_generation_difference = 100,
#     random_generation_max = 100, # wind like 
#     beta_S = 10, 
#     linespace_expansion = 1)#(N_treatment + N_control)) # if > 1, significantly increase generation time 

# ts_gen_eq.add_multiple_demand_functions(N=N_treatment, d_0=10, beta_P=-.5, noise_variance_demand = 10)
# ts_gen_eq.add_multiple_demand_functions(N=N_control, d_0=10, beta_P=0, noise_variance_demand = 10)

# # Generate equilibrium time series with a dummy seasonality function
# ts_gen_eq.generate_time_series()

### with functionality of expected load

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
 
class EquilibriumTimeSeriesGeneratorIncludingExpectedPriceResponse:
    def __init__(
        self,
        T=100,
        T_expectation_horizon = 100, 
        S_0=0,
        beta_capacity=1, 
        seasonal_generation_difference=100, 
        seasonal_generation_variance = 10, 
        beta_S=10, 
        generation_noise_variance=1, 
        random_generation_max=10, 
        seasonal_generation_function=None, 
        linespace_expansion=0, 
        T_initialization=200, 
        print_milestones = True):
        self.T = T
        self.T_initialization = T_initialization
        self.T_expectation_horizon = T_expectation_horizon, 
        self.S_0 = S_0
        self.beta_capacity = beta_capacity
        self.seasonal_generation_difference = seasonal_generation_difference
        self.seasonal_generation_function = seasonal_generation_function
        self.seasonal_generation_variance = seasonal_generation_variance
        self.random_generation_max = random_generation_max
        self.beta_S = beta_S
        self.generation_noise_variance = generation_noise_variance
        self.demand_functions = []
        self.individual_noises = []  # To store noise series for each demand function
        self.seasonal_demand_series = []
        self.df = None
        self.linespace_expansion = linespace_expansion
        self.print_milestones = print_milestones
 
    def demand_func(self, P_t, d_0, beta_P, P_exp_t, beta_EXP=0, epsilon_D=0, seasonal_load_difference_t=0):
        return d_0 + beta_P * P_t + seasonal_load_difference_t + epsilon_D + beta_EXP * P_exp_t

    def add_demand_function(self, d_0, beta_P, noise_variance_demand, seasonal_load_difference=0, seasonal_load_function=None, beta_EXP=0):
        demand_noise = np.random.normal(0, noise_variance_demand, self.T + self.T_initialization)
        if seasonal_load_function is not None:
            seasonal_load = seasonal_load_function(np.arange(self.T + self.T_initialization))
            seasonal_demand = seasonal_load * seasonal_load_difference
        else:
            seasonal_demand = np.zeros(self.T + self.T_initialization)
        self.individual_noises.append(demand_noise)
        self.seasonal_demand_series.append(seasonal_demand)
        # Ensure the lambda captures beta_EXP correctly
        self.demand_functions.append(lambda P_t, P_exp_t, seasonal_load_difference_t, epsilon_D, d_0=d_0, beta_P=beta_P, beta_EXP=beta_EXP:
                            self.demand_func(P_t, d_0, beta_P, P_exp_t, beta_EXP, epsilon_D, seasonal_load_difference_t))
 
    def add_multiple_demand_functions(self, N, d_0, beta_P, noise_variance_demand, seasonal_load_difference=0, seasonal_load_function=None, beta_EXP=0):
        for n in range(1, 1 + N):
            self.add_demand_function(d_0, beta_P, noise_variance_demand, seasonal_load_difference, seasonal_load_function, beta_EXP)
 
    def generate_time_series(self):

# Check and correct T_expectation_horizon to ensure it's an integer
        if isinstance(self.T_expectation_horizon, tuple):
            self.T_expectation_horizon = self.T_expectation_horizon[0]
        self.T_expectation_horizon = int(self.T_expectation_horizon)
        
        # Debug: Ensure T_expectation_horizon is correct
        #print(f'Debug: self.T_expectation_horizon is {self.T_expectation_horizon}, type: {type(self.T_expectation_horizon)}')
                # Debug: Ensure T_expectation_horizon is correct
        #print(f'Debug: self.T_expectation_horizon is {self.T_expectation_horizon}, type: {type(self.T_expectation_horizon)}')
        index = np.arange(self.T + self.T_initialization)
        P_eq = []
        S_t_values = []
        D_t_values = []
        individual_D_t_values = {f'D{i+1}_t': [] for i in range(len(self.demand_functions))}
        if self.seasonal_generation_function is not None:
            seasonality_generation = np.array([self.seasonal_generation_function(i) for i in range(self.T + self.T_initialization)])
        else:
            seasonality_generation = np.zeros(self.T + self.T_initialization)

        epsilon_c_t = np.random.random(self.T + self.T_initialization)
        epsilon_c_t_deterministic = np.random.normal(0, self.seasonal_generation_variance, self.T + self.T_initialization)
        C_t = self.seasonal_generation_difference * seasonality_generation + epsilon_c_t_deterministic * seasonality_generation + epsilon_c_t * self.random_generation_max
        
        aggregated_seasonality_load = np.zeros(self.T + self.T_initialization)

        # define grid
        linespace_min = -100 * (1 + self.linespace_expansion)
        linespace_max = 100 * (1 + self.linespace_expansion)
        steps = int((linespace_max - linespace_min) * 500)

        P_exp = np.zeros(self.T + self.T_initialization)
 
        milestones = [int((i/10) * (self.T + self.T_initialization)) for i in range(1, 11)]

                # Check and set T_expectation_horizon
        if self.T_expectation_horizon > 2*self.T_initialization:
            print('Warning: T_expectation_horizon is greater than 2 * T_initialization. Setting T_initialization to 2* T_expectation_horizon to make sure that at least one entire expectation horizon is discarded.')
            self.T_initialization = 2* self.T_expectation_horizon

        for t in range(self.T + self.T_initialization):

            if self.print_milestones is True: 
                if t in milestones:
                    percent_complete = int((t / (self.T + self.T_initialization)) * 100)
                    print(f'{percent_complete} percent of equilibrium finding ready')


            aggregated_seasonality_load[t] = sum([seasonal_load[t] for seasonal_load in self.seasonal_demand_series])
            seasonality_price = seasonality_generation[t] + aggregated_seasonality_load[t]

            # Use T_expectation_horizon instead of T_initialization for matching period
            if t >= self.T_expectation_horizon:
                matching_start = max(0, t - self.T_expectation_horizon)
                matching_indices = np.where(seasonality_generation[matching_start:t] + aggregated_seasonality_load[matching_start:t] == seasonality_price)[0]
                if len(matching_indices) > 0:
                    P_exp[t] = np.nanmean(np.array(P_eq)[matching_start:t][matching_indices])

            # if t >= self.T_initialization: 
            #     matching_start = max(0, t - self.T_initialization)
            #     matching_indices = np.where(seasonality_generation[matching_start:t] + aggregated_seasonality_load[matching_start:t] == seasonality_price)[0]
            #     if len(matching_indices) > 0:
            #         P_exp[t] = np.nanmean(np.array(P_eq)[matching_start:t][matching_indices])



            P_t = np.linspace(linespace_min, linespace_max, steps)
            epsilon_s_t = np.random.normal(0, self.generation_noise_variance)

            def supply_function(P_t):
                return self.S_0 + self.beta_S * P_t + epsilon_s_t + self.beta_capacity * C_t[t]

            def demand_function(P_t, expected_price):
                D_agg_t = np.zeros_like(P_t)
                time_step_demands = []
                for i, (demand_func, seasonal_demand_series, noise_series) in enumerate(zip(self.demand_functions, self.seasonal_demand_series, self.individual_noises)):
                    D_t_i = demand_func(P_t, expected_price, seasonal_load_difference_t=seasonal_demand_series[t], epsilon_D=noise_series[t])
                    time_step_demands.append(D_t_i)
                    D_agg_t += D_t_i
                return D_agg_t, time_step_demands

            supply_values = supply_function(P_t)
            #P_exp[t] = 0 for all 
            demand_values, individual_demands = demand_function(P_t, P_exp[t])
            
            equilibrium_indices = np.where(np.isclose(supply_values, demand_values, atol=1))[0]
            if len(equilibrium_indices) > 0:
                middle_index = (len(equilibrium_indices) // 2)
                P_equilibrium = P_t[equilibrium_indices[middle_index]]
            else:
                P_equilibrium = np.nan
            P_eq.append(P_equilibrium)
            if np.isnan(P_equilibrium):
                S_t_values.append(np.nan)
                D_t_values.append(np.nan)
                for i in range(len(self.demand_functions)):
                    individual_D_t_values[f'D{i+1}_t'].append(np.nan)
            else:
                S_t_values.append(supply_function(P_equilibrium))
                D_t_values.append(demand_values[equilibrium_indices[middle_index]])
                for i in range(len(self.demand_functions)):
                    individual_D_t_values[f'D{i+1}_t'].append(individual_demands[i][equilibrium_indices[middle_index]])
            
            aggregated_seasonality_load[t] = sum([seasonal_load[t] for seasonal_load in self.seasonal_demand_series])
            seasonality_price = seasonality_generation[t] + aggregated_seasonality_load[t]

        for key in individual_D_t_values:
            individual_D_t_values[key] = np.array(individual_D_t_values[key])

        self.df = pd.DataFrame({
            'index': index[self.T_initialization:],
            'seasonality_generation': seasonality_generation[self.T_initialization:],
            'aggregated_seasonality_load': aggregated_seasonality_load[self.T_initialization:],
            'seasonality_price': (seasonality_generation + aggregated_seasonality_load)[self.T_initialization:],
            'C_t': C_t[self.T_initialization:],
            'P_eq': P_eq[self.T_initialization:],
            'S_t': S_t_values[self.T_initialization:],
            'D_agg_t': D_t_values[self.T_initialization:],
            'P_exp': P_exp[self.T_initialization:],
        })

        for key, values in individual_D_t_values.items():
            self.df[key] = values[self.T_initialization:]





### panel data function

In [ ]:
def generate_panel_from_generated_column_data(df, list_N, list_names, base_category=None):
    # Initialize empty DataFrame
    df_panel = pd.DataFrame()

    # Assuming the original df has a continuous index starting from 0
    df.reset_index(drop=True, inplace=True)

    # Create group ranges based on list_N
    group_ranges = []
    start = 1  # Assuming the first group's data starts in the first column
    for N in list_N:
        group_ranges.append(range(start, start + N))
        start += N

    # Collect all temp_df to concatenate them at once
    temp_dfs = []
    for group_range, group_name in zip(group_ranges, list_names):
        for i in group_range:
            col_name = f'D{i}_t'
            if col_name in df.columns:
                temp_df = pd.DataFrame({
                    'load': df[col_name],
                    'FE': f'D{i}',
                    'consumer_type': group_name,
                    'time': df.index
                })
                temp_dfs.append(temp_df)

    # Concatenate all temp_dfs at once to create df_panel
    if temp_dfs:
        df_panel = pd.concat(temp_dfs, ignore_index=True)

    # Add other variables to the panel DataFrame
    all_columns = df.columns.tolist()
    other_vars = [var for var in all_columns if not (var.startswith('D') and var[1:-2].isdigit()) and var != 'index']

    # Create a dataframe containing only the other variables and the time index
    df_other_vars = df[other_vars].copy()
    df_other_vars.reset_index(inplace=True)
    df_other_vars.rename(columns={'index': 'time'}, inplace=True)

    # Ensure 'time' column in df_panel matches the 'time' column in df_other_vars
    df_panel['time'] = df_panel['time'].astype(int)
    df_other_vars['time'] = df_other_vars['time'].astype(int)

    # Merge the other variables dataframe with df_panel on 'time'
    df_panel = pd.merge(df_panel, df_other_vars, on='time', how='left')

    # Convert consumer_type to category and set base category
    if base_category and 'consumer_type' in df_panel.columns:
        df_panel['consumer_type'] = df_panel['consumer_type'].astype('category')
        remaining_categories = [cat for cat in list_names if cat != base_category]
        df_panel['consumer_type'] = df_panel['consumer_type'].cat.reorder_categories([base_category] + remaining_categories, ordered=True)

    return df_panel

## generate data

### main data

In [ ]:
generator = EquilibriumTimeSeriesGeneratorIncludingExpectedPriceResponse(
    T=10000,
    S_0=0,
    beta_capacity=1,
    beta_S=10,
    generation_noise_variance=1,
    T_expectation_horizon = 1000, 
    T_initialization=2000, 

    random_generation_max = 100,
    seasonal_generation_difference=100,
    seasonal_generation_variance = 10, 
    seasonal_generation_function=lambda t: 1 if (t % 3) == 1 else 0,
    linespace_expansion=0,
)

# -------------no seasonality 

N_group1 = 10
name_group1 = 'load_actual_price_response'
effect_group1 = -0.2
# load reacting to actual price 
generator.add_multiple_demand_functions(
    N=N_group1,
    d_0=10,
    beta_P=effect_group1,
    noise_variance_demand=10,
    beta_EXP=0
)

N_group2 = 10
name_group2 = 'load_expected_price_response'
effect_group2 = -0.2
# load reacting to expected price 
generator.add_multiple_demand_functions(
    N=N_group2,
    d_0=10,
    beta_P = 0, 
    noise_variance_demand=10,
    beta_EXP=effect_group2
)

N_group3 =  10
name_group3 = 'load_no_response'
effect_group3 = 0
# load not reacting to any price 
generator.add_multiple_demand_functions(
    N=N_group3,
    d_0=10,
    beta_P = effect_group3, 
    noise_variance_demand=10,
    beta_EXP=0
)
 
generator.generate_time_series()

print(f'Number of NaN entries in dataframe {generator.df.columns[generator.df.isna().any()].tolist()}')

N_consumers = N_group1+ N_group2+ N_group3
group_effects = [effect_group1, effect_group2, effect_group3]
group_N = [N_group1, N_group2, N_group3]
group_names = [name_group1, name_group2, name_group3]


## analyse time series

In [ ]:
#variables = ['P_eq', 'P_exp', 'C_t','seasonality_generation','D_agg_t']
#variables = ['P_eq', 'P_exp', 'D1_t', 'D12_t', 'D23_t']
variables = ['P_eq', 'P_exp']

plot_timeseries(df = generator.df, variables= variables)

## estimation fucntions

### ols

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

def estimate_price_reaction_with_ols(df, models):
    summaries = []
    for model in models:
        result = ols(model, data=df).fit()
        summary_dict = {
            'Model': model,
            'Coefficients': result.params.to_dict(),
            'P-values': result.pvalues.to_dict(),
            'Standard Errors': result.bse.to_dict(),  # Corrected attribute
            'Lower CI': result.conf_int().iloc[:, 0].to_dict(),
            'Upper CI': result.conf_int().iloc[:, 1].to_dict()
        }
        summaries.append(summary_dict)
    return summaries


#### IV

In [ ]:

from linearmodels.iv import IV2SLS
import matplotlib.pyplot as plt
import pandas as pd
import re

# TODO fix or at least understand why
import warnings

# Suppress specific warnings related to invalid values and divide by zero
np.seterr(divide='ignore', invalid='ignore')

# Optionally suppress warnings using the warnings module as well
warnings.simplefilter(action='ignore', category=RuntimeWarning)

def estimate_price_reaction_with_IV(df, iv_models):
    summaries = []
    for model in iv_models:
        result = IV2SLS.from_formula(model, data=df).fit()
        summary_dict = {
            'Model': model,
            'F-statistic': result.f_statistic.stat if result.f_statistic is not None else None,
            'Coefficients': result.params.to_dict(),
            'P-values': result.pvalues.to_dict(),
            'Standard Errors': result.std_errors.to_dict(),
            'Lower CI': result.conf_int().iloc[:, 0].to_dict(),
            'Upper CI': result.conf_int().iloc[:, 1].to_dict()
        }
        summaries.append(summary_dict)
    return summaries
# def estimate_price_reaction_with_IV(df, iv_models):
#     if df is None or df.empty:
#         raise ValueError("Dataframe is empty or not initialized.")

#     summaries = []
#     for model in iv_models:
#         try:
#             result = IV2SLS.from_formula(model, data=df).fit()
#             summary_dict = {
#                 'Model': model,
#                 'F-statistic': result.f_statistic.stat if result.f_statistic is not None else None,
#                 'Coefficients': result.params.to_dict(),
#                 'P-values': result.pvalues.to_dict(),
#                 'Standard Errors': result.std_errors.to_dict(),
#                 'Lower CI': result.conf_int().iloc[:, 0].to_dict(),
#                 'Upper CI': result.conf_int().iloc[:, 1].to_dict()
#             }
#             summaries.append(summary_dict)
#         except Exception as e:
#             print(f"An error occurred while fitting the model: {model}")
#             print(f"Error: {e}")
#     return summaries



### coeff visualization

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import re
import numpy as np

def plot_coefficients_with_error_bars(summaries, list_N, list_effects, model_specification=None):
    coeffs = []
    lower_ci = []
    upper_ci = []
    models = []
    colors = []
    markers = []

    # Define color palette and marker styles for more than two groups
    color_palette = ['blue', 'red', 'green', 'blue', 'red', 'green', 'blue', 'red', 'green', 'blue', 'red', 'green']
    marker_styles = ['o', 's', '^', 'D', '*', 'x']  # Circle, Square, Triangle, Diamond, Star, Cross
    linestyle = ['-', '--', '-.', ':', '-', '--']

    # Create group_ranges based on list_N
    group_ranges = []
    start = 1  # Assuming the first group's data starts in the second column
    for N in list_N:
        group_ranges.append(range(start, start + N))
        start += N

    # Extracting coefficients and confidence intervals
    for summary in summaries[1:]:  # Skip the first model's results
        for var_name, coeff in summary['Coefficients'].items():
            if var_name == 'P_eq':
                coeffs.append(coeff)
                lower_ci.append(summary['Lower CI'][var_name])
                upper_ci.append(summary['Upper CI'][var_name])
                models.append(summary['Model'])
                model_index = int(re.search(r'\d+', summary['Model']).group())
                # Assign colors and markers based on group ranges
                for i, group_range in enumerate(group_ranges):
                    if model_index in group_range:
                        colors.append(color_palette[i % len(color_palette)])
                        markers.append(marker_styles[i % len(marker_styles)])
                        break

    # Debugging output: print the lengths of the lists
    print(f"Lengths: coeffs={len(coeffs)}, lower_ci={len(lower_ci)}, upper_ci={len(upper_ci)}, models={len(models)}, colors={len(colors)}, markers={len(markers)}")

    # Check if all lists have the same length
    if not (len(coeffs) == len(lower_ci) == len(upper_ci) == len(models) == len(colors) == len(markers)):
        print("Error: Lists have inconsistent lengths.")
        return

    # Create a dataframe
    plot_df = pd.DataFrame({
        'Model': models,
        'Coefficient': coeffs,
        'LowerCI': lower_ci,
        'UpperCI': upper_ci,
        'Color': colors,
        'Marker': markers
    })

    plot_df['ShortModel'] = plot_df['Model'].apply(lambda x: x.split('_')[0])

    # Sort the dataframe by numerical value extracted from 'Model'
    plot_df['ModelIndex'] = plot_df['Model'].apply(lambda x: int(re.search(r'\d+', x).group()))
    plot_df = plot_df.sort_values('ModelIndex')

    # Plotting
    fig, ax = plt.subplots(figsize=(10, 6))
    for idx, row in plot_df.iterrows():
        yerr = [[row['Coefficient'] - row['LowerCI']], [row['UpperCI'] - row['Coefficient']]]
        if yerr[0][0] < 0 or yerr[1][0] < 0:
            continue
        ax.errorbar(row['ShortModel'], row['Coefficient'],
                    yerr=yerr,
                    fmt=row['Marker'], color=row['Color'], label='_nolegend_')

    # Adding horizontal dashed lines for effects with group colors and styles
    for i, (effect, color) in enumerate(zip(list_effects, color_palette[:len(list_effects)])):
        ax.axhline(effect, color=color, linestyle=linestyle[i % len(linestyle)], linewidth=1)

    # Adding legend for groups
    from matplotlib.lines import Line2D
    legend_elements = [Line2D([0], [0], marker=marker_styles[i % len(marker_styles)], color='w', 
                              markerfacecolor=color_palette[i % len(color_palette)], markersize=10, 
                              linestyle=linestyle[i % len(linestyle)], linewidth=1, label=f'Group {i+1}')
                       for i in range(len(list_N))]
    ax.legend(handles=legend_elements, loc='upper left')

    ax.set_ylabel('Coefficient of P_eq')
    if model_specification:
        ax.set_title(f'Coefficients of P_eq with Error Bars \n{model_specification}')
    else:
        ax.set_title(f'Coefficients of P_eq with Error Bars')

    plt.xticks()  # rotation=90)
    plt.tight_layout()
    plt.show()

## Estimate reaction to the Day-Ahead-Price

### seasonality only in the price

#### without controls

##### OLS  

In [ ]:
# -----------estimation 
models = [
    f"D_agg_t ~ P_eq",
]



for n in range(1, N_consumers + 1):
    models.append(f"D{n}_t ~ P_eq")


ols_results = estimate_price_reaction_with_ols(generator.df, models)
plot_coefficients_with_error_bars(ols_results, group_N, group_effects, "OLS: D{n}_t ~ P_eq")

# Call the estimate_price_reaction function
#estimate_price_reaction_with_ols_with_stargazer_output(df=generator.df, models=models, output='html') 
#estimate_price_reaction_with_ols_with_stargazer_output(df=ts_gen_eq.df, models=models, output='latex')


##### IV  

In [ ]:
# -----------estimation 
IV_models = [
    f"D_agg_t ~ 1+ [P_eq ~ C_t]",
]


for n in range(1, N_consumers + 1):
    IV_models.append(f"D{n}_t ~ 1+ [P_eq ~ C_t]")

# Call the new IV regression function
iv_reg_results_generator = estimate_price_reaction_with_IV(df=generator.df, iv_models=IV_models)


# Example usage
# summaries, treatment_range, control_range are to be defined based on your data.
plot_coefficients_with_error_bars(iv_reg_results_generator, group_N, group_effects, "IV: D{n}_t ~ 1+ [P_eq ~ C_t]")
#TODO: calculate the average real effect of the response ot expected load 
# TODO: calculate the weighted mean per group 

In [ ]:

#different instrument 
# # -----------estimation 
IV_models = [
    f"D_agg_t ~ 1+ [P_eq ~ seasonality_generation]",
]


for n in range(1, N_consumers + 1):
    IV_models.append(f"D{n}_t ~ 1+ [P_eq ~ seasonality_generation]")

# Call the new IV regression function
iv_reg_results_generator = estimate_price_reaction_with_IV(df=generator.df, iv_models=IV_models)


# Example usage
# summaries, treatment_range, control_range are to be defined based on your data.
plot_coefficients_with_error_bars(iv_reg_results_generator, group_N, group_effects, "IV: D{n}_t ~ 1+ [P_eq ~ seasonality_generation]")
#TODO: calculate the average real effect of the response ot expected load 

##### panel 

In [ ]:

base_category =  name_group3

df_panel = generate_panel_from_generated_column_data(generator.df, group_N, group_names, base_category)

fit = pf.feols('load ~ consumer_type * P_eq | csw0(FE)', data = df_panel,  vcov = {'CRV1':'FE'})
fit.etable()

#### with seasonality controls

##### OLS  

In [ ]:
# -----------estimation 
models = [
    f"D_agg_t ~ seasonality_generation + P_eq",
]



for n in range(1, N_consumers + 1):
    models.append(f"D{n}_t ~ seasonality_generation + P_eq")


ols_results = estimate_price_reaction_with_ols(generator.df, models)
plot_coefficients_with_error_bars(ols_results, group_N, group_effects, "D{n}_t ~ seasonality_generation + P_eq")


##### IV  

In [ ]:
# -----------estimation 
IV_models = [
    f"D_agg_t ~ 1+ seasonality_generation + [P_eq ~ C_t]",
]


for n in range(1, N_consumers + 1):
    IV_models.append(f"D{n}_t ~ 1 + seasonality_generation +  [P_eq ~ C_t]")

# Call the new IV regression function
iv_reg_results_generator = estimate_price_reaction_with_IV(df=generator.df, iv_models=IV_models)


# Example usage
# summaries, treatment_range, control_range are to be defined based on your data.
plot_coefficients_with_error_bars(iv_reg_results_generator, group_N, group_effects, "D{n}_t ~ 1 + seasonality_generation +  [P_eq ~ C_t]")
#TODO: calculate the average real effect of the response ot expected load 

##### panel

In [ ]:

base_category =  name_group3

df_panel = generate_panel_from_generated_column_data(generator.df, group_N, group_names, base_category)

fit = pf.feols('load ~  C(seasonality_generation) + consumer_type *P_eq | csw0(FE)', data = df_panel,  vcov = {'CRV1':'FE'})
fit.etable()

### load with seasonality 

In [ ]:
# TODO: make to to have the genration_response as a dummy variable, and not as a continous(partiuclarly aggregated seasonality)

In [ ]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
 
# class EquilibriumTimeSeriesGeneratorIncludingExpectedPriceResponse_WIP:
#     def __init__(
#         self,
#         T=100,
#         T_expectation_horizon = 100, 
#         S_0=0,
#         beta_capacity=1, 
#         seasonal_generation_difference=100, 
#         seasonal_generation_variance = 10, 
#         beta_S=10, 
#         generation_noise_variance=1, 
#         random_generation_max=10, 
#         seasonal_generation_function=None, 
#         linespace_expansion=0, 
#         T_initialization=200):
#         self.T = T
#         self.T_initialization = T_initialization
#         self.T_expectation_horizon = T_expectation_horizon, 
#         self.S_0 = S_0
#         self.beta_capacity = beta_capacity
#         self.seasonal_generation_difference = seasonal_generation_difference
#         self.seasonal_generation_function = seasonal_generation_function
#         self.seasonal_generation_variance = seasonal_generation_variance
#         self.random_generation_max = random_generation_max
#         self.beta_S = beta_S
#         self.generation_noise_variance = generation_noise_variance
#         self.demand_functions = []
#         self.individual_noises = []  # To store noise series for each demand function
#         self.seasonal_demand_series = []
#         self.df = None
#         self.linespace_expansion = linespace_expansion
 
#     def demand_func(self, P_t, d_0, beta_P, P_exp_t, beta_EXP=0, epsilon_D=0, seasonal_load_difference_t=0):
#         return d_0 + beta_P * P_t + seasonal_load_difference_t + epsilon_D + beta_EXP * P_exp_t

#     def add_demand_function(self, d_0, beta_P, noise_variance_demand, seasonal_load_difference=0, seasonal_load_function=None, beta_EXP=0):
#         demand_noise = np.random.normal(0, noise_variance_demand, self.T + self.T_initialization)
#         if seasonal_load_function is not None:
#             seasonal_load = seasonal_load_function(np.arange(self.T + self.T_initialization))
#             seasonal_demand = seasonal_load * seasonal_load_difference
#         else:
#             seasonal_demand = np.zeros(self.T + self.T_initialization)
#         self.individual_noises.append(demand_noise)
#         self.seasonal_demand_series.append(seasonal_demand)
#         # Ensure the lambda captures beta_EXP correctly
#         self.demand_functions.append(lambda P_t, P_exp_t, seasonal_load_difference_t, epsilon_D, d_0=d_0, beta_P=beta_P, beta_EXP=beta_EXP:
#                             self.demand_func(P_t, d_0, beta_P, P_exp_t, beta_EXP, epsilon_D, seasonal_load_difference_t))
 
#     def add_multiple_demand_functions(self, N, d_0, beta_P, noise_variance_demand, seasonal_load_difference=0, seasonal_load_function=None, beta_EXP=0):
#         for n in range(1, 1 + N):
#             self.add_demand_function(d_0, beta_P, noise_variance_demand, seasonal_load_difference, seasonal_load_function, beta_EXP)
 
#     def generate_time_series(self):

# # Check and correct T_expectation_horizon to ensure it's an integer
#         if isinstance(self.T_expectation_horizon, tuple):
#             self.T_expectation_horizon = self.T_expectation_horizon[0]
#         self.T_expectation_horizon = int(self.T_expectation_horizon)
        
#         # Debug: Ensure T_expectation_horizon is correct
#         #print(f'Debug: self.T_expectation_horizon is {self.T_expectation_horizon}, type: {type(self.T_expectation_horizon)}')
#                 # Debug: Ensure T_expectation_horizon is correct
#         #print(f'Debug: self.T_expectation_horizon is {self.T_expectation_horizon}, type: {type(self.T_expectation_horizon)}')
#         index = np.arange(self.T + self.T_initialization)
#         P_eq = []
#         S_t_values = []
#         D_t_values = []
#         individual_D_t_values = {f'D{i+1}_t': [] for i in range(len(self.demand_functions))}
#         if self.seasonal_generation_function is not None:
#             seasonality_generation = np.array([self.seasonal_generation_function(i) for i in range(self.T + self.T_initialization)])
#         else:
#             seasonality_generation = np.zeros(self.T + self.T_initialization)

#         epsilon_c_t = np.random.random(self.T + self.T_initialization)
#         epsilon_c_t_deterministic = np.random.normal(0, self.seasonal_generation_variance, self.T + self.T_initialization)
#         C_t = self.seasonal_generation_difference * seasonality_generation + epsilon_c_t_deterministic * seasonality_generation + epsilon_c_t * self.random_generation_max
        
#         aggregated_seasonality_load = np.zeros(self.T + self.T_initialization)

#         # define grid
#         linespace_min = -100 * (1 + self.linespace_expansion)
#         linespace_max = 100 * (1 + self.linespace_expansion)
#         steps = int((linespace_max - linespace_min) * 500)

#         P_exp = np.zeros(self.T + self.T_initialization)
#         milestones = [int((i/10) * (self.T + self.T_initialization)) for i in range(1, 11)]


#                 # Check and set T_expectation_horizon
#         if self.T_expectation_horizon > 2*self.T_initialization:
#             print('Warning: T_expectation_horizon is greater than 2 * T_initialization. Setting T_initialization to 2* T_expectation_horizon to make sure that at least one entire expectation horizon is discarded.')
#             self.T_initialization = 2* self.T_expectation_horizon

#         for t in range(self.T + self.T_initialization):


#             if t in milestones:
#                 percent_complete = int((t / (self.T + self.T_initialization)) * 100)
#                 print(f'{percent_complete} percent of equilibrium finding ready')


#             aggregated_seasonality_load[t] = sum([seasonal_load[t] for seasonal_load in self.seasonal_demand_series])
#             seasonality_price = seasonality_generation[t] + aggregated_seasonality_load[t]

#             # Use T_expectation_horizon instead of T_initialization for matching period
#             if t >= self.T_expectation_horizon:
#                 matching_start = max(0, t - self.T_expectation_horizon)
#                 matching_indices = np.where(seasonality_generation[matching_start:t] + aggregated_seasonality_load[matching_start:t] == seasonality_price)[0]
#                 if len(matching_indices) > 0:
#                     P_exp[t] = np.nanmean(np.array(P_eq)[matching_start:t][matching_indices])

#             # if t >= self.T_initialization: 
#             #     matching_start = max(0, t - self.T_initialization)
#             #     matching_indices = np.where(seasonality_generation[matching_start:t] + aggregated_seasonality_load[matching_start:t] == seasonality_price)[0]
#             #     if len(matching_indices) > 0:
#             #         P_exp[t] = np.nanmean(np.array(P_eq)[matching_start:t][matching_indices])



#             P_t = np.linspace(linespace_min, linespace_max, steps)
#             epsilon_s_t = np.random.normal(0, self.generation_noise_variance)

#             def supply_function(P_t):
#                 return self.S_0 + self.beta_S * P_t + epsilon_s_t + self.beta_capacity * C_t[t]

#             def demand_function(P_t, expected_price):
#                 D_agg_t = np.zeros_like(P_t)
#                 time_step_demands = []
#                 for i, (demand_func, seasonal_demand_series, noise_series) in enumerate(zip(self.demand_functions, self.seasonal_demand_series, self.individual_noises)):
#                     D_t_i = demand_func(P_t, expected_price, seasonal_load_difference_t=seasonal_demand_series[t], epsilon_D=noise_series[t])
#                     time_step_demands.append(D_t_i)
#                     D_agg_t += D_t_i
#                 return D_agg_t, time_step_demands

#             supply_values = supply_function(P_t)
#             #P_exp[t] = 0 for all 
#             demand_values, individual_demands = demand_function(P_t, P_exp[t])
            
#             equilibrium_indices = np.where(np.isclose(supply_values, demand_values, atol=1))[0]
#             if len(equilibrium_indices) > 0:
#                 middle_index = (len(equilibrium_indices) // 2)
#                 P_equilibrium = P_t[equilibrium_indices[middle_index]]
#             else:
#                 P_equilibrium = np.nan
#             P_eq.append(P_equilibrium)
#             if np.isnan(P_equilibrium):
#                 S_t_values.append(np.nan)
#                 D_t_values.append(np.nan)
#                 for i in range(len(self.demand_functions)):
#                     individual_D_t_values[f'D{i+1}_t'].append(np.nan)
#             else:
#                 S_t_values.append(supply_function(P_equilibrium))
#                 D_t_values.append(demand_values[equilibrium_indices[middle_index]])
#                 for i in range(len(self.demand_functions)):
#                     individual_D_t_values[f'D{i+1}_t'].append(individual_demands[i][equilibrium_indices[middle_index]])
            
#             aggregated_seasonality_load[t] = sum([seasonal_load[t] for seasonal_load in self.seasonal_demand_series])
#             seasonality_price = seasonality_generation[t] + aggregated_seasonality_load[t]

#         for key in individual_D_t_values:
#             individual_D_t_values[key] = np.array(individual_D_t_values[key])

#         self.df = pd.DataFrame({
#             'index': index[self.T_initialization:],
#             'seasonality_generation': seasonality_generation[self.T_initialization:],
#             'aggregated_seasonality_load': aggregated_seasonality_load[self.T_initialization:],
#             'seasonality_price': (seasonality_generation + aggregated_seasonality_load)[self.T_initialization:],
#             'C_t': C_t[self.T_initialization:],
#             'P_eq': P_eq[self.T_initialization:],
#             'S_t': S_t_values[self.T_initialization:],
#             'D_agg_t': D_t_values[self.T_initialization:],
#             'P_exp': P_exp[self.T_initialization:],
#         })

#         for key, values in individual_D_t_values.items():
#             self.df[key] = values[self.T_initialization:]





#### generate the seasonal data 

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
# TODO fix error causing warning 

seasonal_generator = EquilibriumTimeSeriesGeneratorIncludingExpectedPriceResponse(
    T=10000,
    S_0=0,
    beta_capacity=1,
    beta_S=10,
    generation_noise_variance=1,
    T_expectation_horizon = 1000, 
    T_initialization=4000, 

    random_generation_max = 400,
    seasonal_generation_difference=400,
    seasonal_generation_variance = 40, 
    seasonal_generation_function=lambda t: 1 if (t % 3) == 1 else 0,
    #add a sesaonal_load_fuction 
    linespace_expansion=0,
)

# no sesaonality ----------------------

N_group1 = 10
name_group1 = 'load_actual_price_response'
effect_group1 = -0.2
# load reacting to actual price 
seasonal_generator.add_multiple_demand_functions(
    N=N_group1,
    d_0=10,
    beta_P=effect_group1,
    noise_variance_demand=10,
    beta_EXP=0
)

N_group2 = 10
name_group2 = 'load_expected_price_response'
effect_group2 = -0.2
# load reacting to expected price 
seasonal_generator.add_multiple_demand_functions(
    N=N_group2,
    d_0=10,
    beta_P = 0, 
    noise_variance_demand=10,
    beta_EXP=effect_group2
)

N_group3 =  10
name_group3 = 'load_no_response'
effect_group3 = 0
# load not reacting to any price 
seasonal_generator.add_multiple_demand_functions(
    N=N_group3,
    d_0=10,
    beta_P = effect_group3, 
    noise_variance_demand=10,
    beta_EXP=0
)

# ------------adding sesaonality 011
N_group4 =  10
name_group4 = 'seasonal_011_load_actual_price_response'
effect_group4 = -0.2
# load not reacting to any price 
seasonal_generator.add_multiple_demand_functions(
    N=N_group4,
    d_0=10,
    beta_P = effect_group4, 
    noise_variance_demand=10,
    beta_EXP=0, 
    seasonal_load_difference = 5, 
    seasonal_load_function = lambda t: np.where((t % 3) < 2, 1, 0) # returns 011 011
)

N_group5 =  10
name_group5 = 'seasonal_011_load_expected_price_response'
effect_group5 = -0.2
# load not reacting to any price 
seasonal_generator.add_multiple_demand_functions(
    N=N_group5,
    d_0=10,
    beta_P = 0, 
    noise_variance_demand=10,
    beta_EXP=effect_group5, 
    seasonal_load_difference = 5, 
    seasonal_load_function = lambda t: np.where((t % 3) < 2, 1, 0) # returns 011 011
)

N_group6 =  10
name_group6 = 'seasonal_011_load_no_repsonse'
effect_group6 = -0.2
# load not reacting to any price 
seasonal_generator.add_multiple_demand_functions(
    N=N_group6,
    d_0=10,
    beta_P = 0, 
    noise_variance_demand=10,
    beta_EXP=effect_group6, 
    seasonal_load_difference = 5, 
    seasonal_load_function = lambda t: np.where((t % 3) < 2, 1, 0) # returns 011 011
)

# -----------different sesaonality load 110
N_group7 =  10
name_group7 = 'seasonal_110_load_actual_price_response'
effect_group7 = -.2
# load not reacting to any price 
seasonal_generator.add_multiple_demand_functions(
    N=N_group7,
    d_0=10,
    beta_P = effect_group7, 
    noise_variance_demand=10,
    beta_EXP=0, 
    seasonal_load_difference = 5, 
    seasonal_load_function = lambda t: np.where((t % 6) < 2, 1, np.where((t % 6) < 4, 1, 0)) #
)

N_group8 =  10
name_group8 = 'seasonal_110_load_actual_price_response'
effect_group8 = -0.2
# load not reacting to any price 
seasonal_generator.add_multiple_demand_functions(
    N=N_group8,
    d_0=10,
    beta_P = 0, 
    noise_variance_demand=10,
    beta_EXP=effect_group8, 
    seasonal_load_difference = 5, 
    seasonal_load_function = lambda t: np.where((t % 6) < 2, 1, np.where((t % 6) < 4, 1, 0))# returns 011 011
)

N_group9 =  10
name_group9 = 'seasonal_110_load_no_response'
effect_group9 = 0
# load not reacting to any price 
seasonal_generator.add_multiple_demand_functions(
    N=N_group9,
    d_0=10,
    beta_P = 0, 
    noise_variance_demand=10,
    beta_EXP=effect_group9, 
    seasonal_load_difference = 5, 
    seasonal_load_function = lambda t: np.where((t % 6) < 2, 1, np.where((t % 6) < 4, 1, 0)) # returns 011 011
)

# -----------same sesaonality as generation 010
N_group10 =  10
name_group10 = 'seasonal_010_load_actual_price_response'
effect_group10 = -0.2
# load not reacting to any price 
seasonal_generator.add_multiple_demand_functions(
    N=N_group10,
    d_0=10,
    beta_P = effect_group10, 
    noise_variance_demand=10,
    beta_EXP=0, 
    seasonal_load_difference = 5, 
    seasonal_load_function = lambda t: np.where(t % 3 == 1, 1, 0)# returns 011 011
)

N_group11 =  10
name_group11 = 'seasonal_010_load_expected_price_response'
effect_group11 = -0.2
# load not reacting to any price 
seasonal_generator.add_multiple_demand_functions(
    N=N_group11,
    d_0=10,
    beta_P = 0, 
    noise_variance_demand=10,
    beta_EXP=effect_group11, 
    seasonal_load_difference = 5, 
    seasonal_load_function = lambda t: np.where(t % 3 == 1, 1, 0)# returns 011 011
)

N_group12 =  10
name_group12 = 'seasonal_010_load_no_response'
effect_group12 = 0
# load not reacting to any price 
seasonal_generator.add_multiple_demand_functions(
    N=N_group12,
    d_0=10,
    beta_P = 0, 
    noise_variance_demand=10,
    beta_EXP=effect_group12, 
    seasonal_load_difference = 5, 
    seasonal_load_function = lambda t: np.where(t % 3 == 1, 1, 0) # returns 011 011
)
 
seasonal_generator.generate_time_series()

print(f'Number of NaN entries in dataframe {seasonal_generator.df.columns[seasonal_generator.df.isna().any()].tolist()}')

N_consumers = N_group1+ N_group2+ N_group3 + N_group4 + N_group5 + N_group6 + N_group7 + N_group8+ N_group9 + N_group10 + N_group11 + N_group12
group_effects = [effect_group1, effect_group2, effect_group3, effect_group4, effect_group5, effect_group6, effect_group7, effect_group8, effect_group9, effect_group10, effect_group11, effect_group12]
group_N = [N_group1, N_group2, N_group3, N_group4, N_group5, N_group6, N_group7, N_group8, N_group9, N_group10 , N_group11 , N_group12]
group_names = [name_group1, name_group2, name_group3, name_group4, name_group5, name_group6, name_group7, name_group8, name_group9, name_group10, name_group11, name_group12]


###### Analyse timeseries

In [ ]:
#variables = ['P_eq', 'P_exp', 'C_t','seasonality_generation','aggregated_seasonality_load','D_agg_t']
variables = ['P_eq', 'P_exp', 'D1_t', 'D2_t', 'D3_t', 'D4_t', 'D5_t', 'D6_t','D7_t', 'D8_t', 'D9_t', 'D10_t', 'D11_t', 'D12_t', 'seasonality_generation', 'aggregated_seasonality_load']
#variables = ['P_eq', 'P_exp']


plot_timeseries(df = seasonal_generator.df, variables= variables)

#### estimate the effect data

In [ ]:
df_to_estimate = seasonal_generator.df
#seasonal_generator.df = df_seasonal_generator_with_noise 


##### without controls

###### OLS

In [ ]:
# -----------estimation 
models = [
    f"D_agg_t ~ P_eq",
]



for n in range(1, N_consumers + 1):
    models.append(f"D{n}_t ~ P_eq")


ols_results = estimate_price_reaction_with_ols(df_to_estimate, models)
plot_coefficients_with_error_bars(ols_results, group_N, group_effects, "OLS: D{n}_t ~ P_eq")


###### IV

In [ ]:
# -----------estimation 
IV_models = [
    f"D_agg_t ~ 1+ [P_eq ~ C_t]",
]


for n in range(1, N_consumers + 1):
    IV_models.append(f"D{n}_t ~ 1+ [P_eq ~ C_t]")

# Call the new IV regression function
iv_reg_results_generator = estimate_price_reaction_with_IV(df_to_estimate, iv_models=IV_models)


# Example usage
# summaries, treatment_range, control_range are to be defined based on your data.
plot_coefficients_with_error_bars(iv_reg_results_generator, group_N, group_effects, "IV: D{n}_t ~ 1+ [P_eq ~ C_t]")
#TODO: calculate the average real effect of the response ot expected load 
# TODO: calculate the weighted mean per group 

In [ ]:

# #different instrument 
# # # -----------estimation 
# IV_models = [
#     f"D_agg_t ~ 1+ [P_eq ~ C(seasonality_generation)]",
# ]


# for n in range(1, N_consumers + 1):
#     IV_models.append(f"D{n}_t ~ 1+ [P_eq ~ C(seasonality_generation)]")

# # Call the new IV regression function
# iv_reg_results_generator = estimate_price_reaction_with_IV(df_to_estimate, iv_models=IV_models)


# # Example usage
# # summaries, treatment_range, control_range are to be defined based on your data.
# plot_coefficients_with_error_bars(iv_reg_results_generator, group_N, group_effects, "IV: D{n}_t ~ 1+ [P_eq ~ C(seasonality_generation)]")
# #TODO: calculate the average real effect of the response ot expected load 

###### panel

In [ ]:

base_category =  name_group3

df_panel = generate_panel_from_generated_column_data(df_to_estimate, group_N, group_names, base_category)

fit = pf.feols('load ~ consumer_type * P_eq | csw0(FE)', data = df_panel,  vcov = {'CRV1':'FE'})
fit.etable()

##### with seasonality controls 

###### OLS

In [ ]:
# -----------estimation 
models = [
    f"D_agg_t ~ C(seasonality_generation) + P_eq",
]



for n in range(1, N_consumers + 1):
    models.append(f"D{n}_t ~ C(seasonality_generation) + P_eq")


ols_results = estimate_price_reaction_with_ols(df_to_estimate, models)
plot_coefficients_with_error_bars(ols_results, group_N, group_effects, "D{n}_t ~ C(seasonality_generation) + P_eq")


In [ ]:
# -----------estimation 
models = [
    f"D_agg_t ~ C(aggregated_seasonality_load) + P_eq",
]



for n in range(1, N_consumers + 1):
    models.append(f"D{n}_t ~  C(aggregated_seasonality_load) + P_eq")


ols_results = estimate_price_reaction_with_ols(df_to_estimate, models)
plot_coefficients_with_error_bars(ols_results, group_N, group_effects, "D{n}_t ~  C(aggregated_seasonality_load) + P_eq")


In [ ]:
# -----------estimation 
models = [
    f"D_agg_t ~ C(seasonality_generation) + C(aggregated_seasonality_load) + P_eq",
]



for n in range(1, N_consumers + 1):
    models.append(f"D{n}_t ~ C(seasonality_generation) + C(aggregated_seasonality_load) + P_eq")


ols_results = estimate_price_reaction_with_ols(df_to_estimate, models)
plot_coefficients_with_error_bars(ols_results, group_N, group_effects, "D{n}_t ~ C(seasonality_generation) + C(aggregated_seasonality_load) + P_eq")


###### IV

In [ ]:
# -----------estimation 
IV_models = [
    f"D_agg_t ~ 1+ C(seasonality_generation) + [P_eq ~ C_t]",
]


for n in range(1, N_consumers + 1):
    IV_models.append(f"D{n}_t ~ 1 + C(seasonality_generation) +  [P_eq ~ C_t]")

# Call the new IV regression function
iv_reg_results_generator = estimate_price_reaction_with_IV(df_to_estimate, iv_models=IV_models)


# Example usage
# summaries, treatment_range, control_range are to be defined based on your data.
plot_coefficients_with_error_bars(iv_reg_results_generator, group_N, group_effects, "D{n}_t ~ 1 + C(seasonality_generation) +  [P_eq ~ C_t]")
#TODO: calculate the average real effect of the response ot expected load 

In [ ]:
# -----------estimation 
IV_models = [
    f"D_agg_t ~ 1+ C(aggregated_seasonality_load) +  [P_eq ~ C_t]",
]


for n in range(1, N_consumers + 1):
    IV_models.append(f"D{n}_t ~ 1 + C(aggregated_seasonality_load)+  [P_eq ~ C_t]")

# Call the new IV regression function
iv_reg_results_generator = estimate_price_reaction_with_IV(df_to_estimate ,iv_models=IV_models)


# Example usage
# summaries, treatment_range, control_range are to be defined based on your data.
plot_coefficients_with_error_bars(iv_reg_results_generator, group_N, group_effects, "D{n}_t ~ 1 + C(aggregated_seasonality_load) + [P_eq ~ C_t]")
#TODO: calculate the average real effect of the response ot expected load 

In [ ]:
# -----------estimation 
IV_models = [
    f"D_agg_t ~ 1+ C(seasonality_generation) + C(aggregated_seasonality_load) +  [P_eq ~ C_t]",
]


for n in range(1, N_consumers + 1):
    IV_models.append(f"D{n}_t ~ 1 + C(seasonality_generation) + C(aggregated_seasonality_load)+  [P_eq ~ C_t]")

# Call the new IV regression function
iv_reg_results_generator = estimate_price_reaction_with_IV(df_to_estimate, iv_models=IV_models)


# Example usage
# summaries, treatment_range, control_range are to be defined based on your data.
plot_coefficients_with_error_bars(iv_reg_results_generator, group_N, group_effects, "D{n}_t ~ 1 + C(seasonality_generation) +  C(aggregated_seasonality_load) + [P_eq ~ C_t]")
#TODO: calculate the average real effect of the response ot expected load 

In [ ]:
# -----------estimation 
IV_models = [
    f"D_agg_t ~ 1+ C(seasonality_generation) + [P_eq ~ C_t]",
]


for n in range(1, N_consumers + 1):
    IV_models.append(f"D{n}_t ~ 1 + C(seasonality_generation) +  [P_eq ~ C_t]")

# Call the new IV regression function
iv_reg_results_generator = estimate_price_reaction_with_IV(df_to_estimate, iv_models=IV_models)


# Example usage
# summaries, treatment_range, control_range are to be defined based on your data.
plot_coefficients_with_error_bars(iv_reg_results_generator, group_N, group_effects, "D{n}_t ~ 1 + C(seasonality_generation) +  [P_eq ~ C_t]")
#TODO: calculate the average real effect of the response ot expected load 

###### panel 

In [ ]:
df_to_estimate.columns

In [ ]:

base_category =  name_group3

df_panel = generate_panel_from_generated_column_data(df_to_estimate, group_N, group_names, base_category)

# Corrected formula, ensuring parentheses and syntax are correct
formula = 'load ~ 1+ consumer_type * P_eq | csw0(C(seasonality_generation), C(aggregated_seasonality_load), FE)'

fit = pf.feols(formula, data = df_panel,  vcov = {'CRV1':'FE'})
fit.etable()

In [ ]:
##  what happens if I add seasonality? 
# 1. seasonality in actual load only 
# 2. seasonality in no response only 
# 3. seasonality in actual laod and no response only
# more than one consumer type?  
# could I add that all to the same regression? 
##  what happens if I add response to expected price? 
## what if I have a panel IV regression? --> don't do that 
# diffferent seasonality in price and demand? 
# what happens if I have two different demand responses
# what if I add a seaonal loadprovidel ? 


##### with aggregated load as control  

###### OLS

In [ ]:
# -----------estimation 
models = [
    f"D_agg_t ~ P_eq",
]



for n in range(1, N_consumers + 1):
    models.append(f"D{n}_t ~ P_eq + D_agg_t")


ols_results = estimate_price_reaction_with_ols(df_to_estimate, models)
plot_coefficients_with_error_bars(ols_results, group_N, group_effects, "OLS: D{n}_t ~ P_eq + D_agg_t")


###### IV

In [ ]:
# -----------estimation 
IV_models = [
    f"D_agg_t ~ 1+ [P_eq ~ C_t]",
]


for n in range(1, N_consumers + 1):
    IV_models.append(f"D{n}_t ~ 1+ D_agg_t + [P_eq ~ C_t]")

# Call the new IV regression function
iv_reg_results_generator = estimate_price_reaction_with_IV(df_to_estimate, iv_models=IV_models)


# Example usage
# summaries, treatment_range, control_range are to be defined based on your data.
plot_coefficients_with_error_bars(iv_reg_results_generator, group_N, group_effects, "IV: D{n}_t ~ 1+D_agg_t+  [P_eq ~ C_t]")
#TODO: calculate the average real effect of the response ot expected load 
# TODO: calculate the weighted mean per group 

###### panel

# When does an OLS regression fail? 


    Background: It is often argued, that if the number of customers that are under a real time contract is small, the effect can be estimated with an OLS regression anyways. See for example the paper by Cramton. So, what is the share of customers above which this is the case? 
    

## generate data

In [ ]:
import warnings
import re
import pandas as pd
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
N_consumers = 100
# Placeholder for data collection
data_results = {
    'coeff': [],
    'standard_error': [],
    'Upper CI': [],
    'Lower CI': [],
    'Di_t': [],
    'N_group1': [], 
    'consumer_type': [],
}
for i in range(1, N_consumers + 1):
    print(i)
    seasonal_generator_N = EquilibriumTimeSeriesGeneratorIncludingExpectedPriceResponse(
        T=10000,
        S_0=0,
        beta_capacity=1,
        beta_S=10,
        generation_noise_variance=1,
        T_expectation_horizon=10,
        T_initialization=20,
        random_generation_max=100,
        seasonal_generation_difference=100,
        seasonal_generation_variance=10,
        seasonal_generation_function=lambda t: 1 if (t % 3) == 1 else 0,
        linespace_expansion=0,
        print_milestones = False,
    )
    # No seasonality
    N_group1 = i
    name_group1 = 'load_actual_price_response'
    effect_group1 = -0.2
    # Load reacting to actual price
    seasonal_generator_N.add_multiple_demand_functions(
        N=N_group1,
        d_0=10,
        beta_P=effect_group1,
        noise_variance_demand=10,
        beta_EXP=0
    )
    N_group3 = N_consumers - N_group1
    name_group3 = 'load_no_response'
    effect_group3 = 0
    # Load not reacting to any price
    seasonal_generator_N.add_multiple_demand_functions(
        N=N_group3,
        d_0=10,
        beta_P=effect_group3,
        noise_variance_demand=10,
        beta_EXP=0
    )
    seasonal_generator_N.generate_time_series()
    models = ["D_agg_t ~ P_eq"]

    for n in range(1, N_consumers + 1):
        models.append(f"D{n}_t ~ P_eq")
    
    ols_results = estimate_price_reaction_with_ols(seasonal_generator_N.df, models)
    for summary in ols_results[1:]:  # Skip the first model's results
        model_index = int(re.search(r'\d+', summary['Model']).group())
        coefficient = summary['Coefficients']['P_eq']
        std_err = summary['Standard Errors']['P_eq']
        lower_ci = summary['Lower CI']['P_eq']
        upper_ci = summary['Upper CI']['P_eq']
        
        data_results['coeff'].append(coefficient)
        data_results['standard_error'].append(std_err)
        data_results['Lower CI'].append(lower_ci)
        data_results['Upper CI'].append(upper_ci)
        data_results['Di_t'].append(f"D{model_index}_t")
        data_results['N_group1'].append(N_group1)

    for n in range(1, N_consumers + 1):
        if n <= N_group1: 
            group = name_group1
        else: 
            group = name_group2
        data_results['consumer_type'].append(group)
    del seasonal_generator_N
# Create the final dataframe
results_df = pd.DataFrame(data_results)
print(results_df)


In [ ]:
test = results_df

In [ ]:

# Assuming df_results is your dataframe
# Calculate the weights
results_df['weights'] = 1 / results_df['standard_error'] ** 2

# Function to calculate weighted mean and confidence interval
def weighted_stats(group):
    weights = group['weights']
    coeff = group['coeff']
    
    weighted_mean = np.sum(weights * coeff) / np.sum(weights)
    weighted_se = np.sqrt(1 / np.sum(weights))
    upper_ci = weighted_mean + 1.96 * weighted_se
    lower_ci = weighted_mean - 1.96 * weighted_se
    
    return pd.Series({
        'weighted_mean': weighted_mean,
        'weighted_se': weighted_se,
        'upper_ci': upper_ci,
        'lower_ci': lower_ci
    })

# Group by 'consumer_type' and 'N_group1' and apply the function
results = results_df.groupby(['consumer_type', 'N_group1']).apply(weighted_stats).reset_index()

# Plot the weighted means with confidence intervals
plt.figure(figsize=(12, 8))
for consumer_type, group in results.groupby('consumer_type'):
    plt.errorbar(group['N_group1'], group['weighted_mean'], 
                 yerr=(group['weighted_mean'] - group['lower_ci'], group['upper_ci'] - group['weighted_mean']), 
                 label=consumer_type, fmt='o-')

plt.xlabel('N_group1')
plt.ylabel('Weighted Mean')
plt.title('Weighted Means per Consumer Type with Confidence Intervals')
plt.legend()
plt.grid(True)
plt.show()